<H2>Caserta GCP Challenge</H2>

Summary: 
  1. query API to get current market JSON & convert to csv file (see Python script)
  2. Import into Cloud Storage (manually using console, could automate) 
  3. Import Cloud Storage file into BigQuery
  4. Do requisite queries

Pull in Coin cap csv file

In [25]:
%%gcs read --object gs://caserta_gcp_challenge/CoinMarketCap.csv --variable cccaps

Ensure it has correct headers and some data...

In [23]:
print(cccaps[:500])

id,name,symbol,rank,price_usd,price_btc,market_cap_usd,available_supply,total_supply,max_supply,percent_change_1h,percent_change_24h,percent_change_7d,last_updated,volume_24h_usd
bitcoin,Bitcoin,BTC,1,9522.89,1.0,161009214964,16907600.0,16907600.0,21000000.0,-4.37,-6.91,-12.12,1520528665,8141870000.0
ethereum,Ethereum,ETH,2,719.519,0.0769281,70558170853.0,98062971.0,98062971.0,,-4.51,-5.27,-17.56,1520528652,2052920000.0
ripple,Ripple,XRP,3,0.842906,0.00009012,32950844859.0,39091956706.0,99992


Python imports before pulling caps csv data from Cloud Storage

In [26]:
from google.datalab import Context
import google.datalab.bigquery as bq
import google.datalab.storage as storage
import pandas as pd
try:
  from StringIO import StringIO
except ImportError:
  from io import BytesIO as StringIO

Import CSV into BigQuery, create schema first

In [30]:
df = pd.read_csv(StringIO(cccaps))
schema = bq.Schema.from_data(df)

Create dataset & new table

In [46]:
bq.Dataset('coin_data').create()
coin_table = bq.Table('coin_data.caps').create(schema = schema, overwrite = True)

Finally, load the csv data

In [47]:
coin_table.load('gs://caserta_gcp_challenge/CoinMarketCap.csv', mode='append',
                  source_format = 'csv', csv_options=bq.CSVOptions(skip_leading_rows = 1))

Job sage-webbing-197320/job_S8n60_vcb7Ob90XPoj0b8ArA_IS0 completed

Ensure data was loaded and looks reasonable (transition into BigQuery, setting up dataset first...)

In [52]:
%%bq query -n coin_data
SELECT * FROM coin_data.caps LIMIT 3

Keep line below separate and use single '%'

In [53]:
%bq execute -q coin_data

id,name,symbol,rank,price_usd,price_btc,market_cap_usd,available_supply,total_supply,max_supply,percent_change_1h,percent_change_24h,percent_change_7d,last_updated,volume_24h_usd
dogecoin,Dogecoin,DOGE,37,0.00406968,4.4e-07,461919021.0,1.13502540888e+11,1.13502540888e+11,,-4.18,-5.68,-34.14,1520528641,19461600.0
monacoin,MonaCoin,MONA,63,4.30056,0.0004598,249106927.0,57924300.0,57924300.0,,-1.68,-6.3,-21.72,1520528642,5006090.0
monero,Monero,XMR,9,297.879,0.0318481,4707145571.0,15802207.0,15802207.0,,-5.2,-12.14,-1.89,1520528642,133431000.0


Do query from 6a) How many coins have a USD price greater than $8,000?

In [54]:
%%bq query -n coin_data
SELECT name, price_usd FROM coin_data.caps
WHERE price_usd > 8000

In [55]:
%bq execute -q coin_data

name,price_usd
Bitcoin,9522.89


Do query from 6b) What is the total market cap of the top 100 cryptocurrencies (in USD)?
NOTE: we know there are only 100, so we won't cheat and do a simple sum

In [56]:
%%bq query -n coin_data
#standardSQL
WITH top100 AS (
  SELECT
    name,
    market_cap_usd
  FROM
    `coin_data.caps`
  LIMIT 100
)
SELECT SUM(market_cap_usd) as sum_market_cap_100 from top100

In [57]:
%bq execute -q coin_data

sum_market_cap_100
3.70714220252e+11


Do query from 6c) Which coins have an available supply less than $5M?

In [58]:
%%bq query -n coin_data
SELECT name, available_supply from coin_data.caps
WHERE available_supply < 5000000

In [59]:
%bq execute -q coin_data

name,available_supply
DigixDAO,2000000.0
Maker,618228.0
Zcash,3435769.0
Byteball Bytes,645222.0
Veritaseum,2036645.0
Genesis Vision,3726781.0
ZCoin,4261755.0


Do query from 6d) Which 5 coins have seen the greatest percentage growth in the last week?

In [60]:
%%bq query -n coin_data
SELECT name, percent_change_7d 
FROM coin_data.caps
ORDER BY percent_change_7d DESC
LIMIT 5

In [61]:
%bq execute -q coin_data

name,percent_change_7d
Nexus,29.68
Genesis Vision,23.34
SIRIN LABS Token,15.02
Iconomi,3.42
Bytom,2.14


Do query from 6e) How many ticker symbols contain the letter "X"?

In [64]:
%%bq query -n coin_data
SELECT COUNT(symbol) as symbol_has_x_count
FROM coin_data.caps
WHERE symbol LIKE '%X%'

In [65]:
%bq execute -q coin_data

symbol_has_x_count
14


Do some cleanup to avoid billing fees...

In [67]:
bq.Dataset('coin_data').delete(delete_contents = True)

Exception: Cannot delete non-existent dataset sage-webbing-197320.coin_data